# **1. Datasets Utilizados**

In [1]:
# Instalar PySpark via pip
!pip install pyspark

In [8]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("satvshr/top-4-used-car-sales-datasets-combined")

print("Path to dataset files:", path)

100%|██████████| 446k/446k [00:00<00:00, 73.7MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/satvshr/top-4-used-car-sales-datasets-combined/versions/2


In [3]:
# Iniciar uma Sessão Spark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ExemploPySpark") \
    .getOrCreate()

## **2. Comparando métodos do pandas x métodos do PySpark**

**Leitura de Dados:**

In [13]:
import pandas as pd
import os

for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        if filename.endswith('.csv'):
            csv_file_path = os.path.join(dirname, filename)
            print(f"Reading CSV file: {csv_file_path}")
            df = pd.read_csv(csv_file_path)
            print(df.head())
            break

Reading CSV file: /root/.cache/kagglehub/datasets/satvshr/top-4-used-car-sales-datasets-combined/versions/2/output.csv
      brand     model transmission  age    fuel  engine       km  owner  \
0  mahindra      thar       manual  4.0  diesel  2184.0  11003.0    1.0   
1   hyundai     verna       manual  6.0  petrol  1591.0  66936.0    1.0   
2      tata   harrier       manual  2.0  diesel  1956.0  27990.0    1.0   
3     honda      city    automatic  1.0  petrol  1498.0   5061.0    1.0   
4      ford  ecosport       manual  3.0  diesel  1498.0  23480.0    1.0   

       price location  mileage  power  seats type  
0  1231000.0      NaN      NaN    NaN    NaN  NaN  
1   786000.0      NaN      NaN    NaN    NaN  NaN  
2  1489000.0      NaN      NaN    NaN    NaN  NaN  
3  1227000.0      NaN      NaN    NaN    NaN  NaN  
4   887000.0      NaN      NaN    NaN    NaN  NaN  


In [19]:
from pyspark.sql import SparkSession
import os

spark = SparkSession.builder.appName("CSVReader").getOrCreate()

for dirname, _, filenames in os.walk(path):
  for filename in filenames:
    if filename.endswith(".csv"):
      csv_file_path = os.path.join(dirname, filename)
      print(f"Reading CSV file: {csv_file_path}")

      df_pyspark = spark.read.csv(csv_file_path, header=True, inferSchema=True)

      df_pyspark.show(5)

      break

Reading CSV file: /root/.cache/kagglehub/datasets/satvshr/top-4-used-car-sales-datasets-combined/versions/2/output.csv
+--------+--------+------------+---+------+------+-------+-----+---------+--------+-------+-----+-----+----+
|   brand|   model|transmission|age|  fuel|engine|     km|owner|    price|location|mileage|power|seats|type|
+--------+--------+------------+---+------+------+-------+-----+---------+--------+-------+-----+-----+----+
|mahindra|    thar|      manual|4.0|diesel|2184.0|11003.0|  1.0|1231000.0|    NULL|   NULL| NULL| NULL|NULL|
| hyundai|   verna|      manual|6.0|petrol|1591.0|66936.0|  1.0| 786000.0|    NULL|   NULL| NULL| NULL|NULL|
|    tata| harrier|      manual|2.0|diesel|1956.0|27990.0|  1.0|1489000.0|    NULL|   NULL| NULL| NULL|NULL|
|   honda|    city|   automatic|1.0|petrol|1498.0| 5061.0|  1.0|1227000.0|    NULL|   NULL| NULL| NULL|NULL|
|    ford|ecosport|      manual|3.0|diesel|1498.0|23480.0|  1.0| 887000.0|    NULL|   NULL| NULL| NULL|NULL|
+--------

In [25]:
# Filtrando colunas com Pandas
selected_columns_pandas = df[['brand', 'fuel']]
selected_columns_pyspark.show()

+--------+------+
|   brand|  fuel|
+--------+------+
|mahindra|diesel|
| hyundai|petrol|
|    tata|diesel|
|   honda|petrol|
|    ford|diesel|
|   honda|petrol|
|    tata|petrol|
|    tata|diesel|
|mahindra|diesel|
|    ford|petrol|
|  maruti|petrol|
| hyundai|petrol|
|    tata|petrol|
| hyundai|petrol|
|  maruti|petrol|
| hyundai|diesel|
|  maruti|petrol|
|   honda|petrol|
|     kia|diesel|
| hyundai|petrol|
+--------+------+
only showing top 20 rows



In [26]:
# Filtrando colunas com PySpark
selected_columns_pyspark = df_pyspark.select('brand', 'fuel')
selected_columns_pyspark.show()

+--------+------+
|   brand|  fuel|
+--------+------+
|mahindra|diesel|
| hyundai|petrol|
|    tata|diesel|
|   honda|petrol|
|    ford|diesel|
|   honda|petrol|
|    tata|petrol|
|    tata|diesel|
|mahindra|diesel|
|    ford|petrol|
|  maruti|petrol|
| hyundai|petrol|
|    tata|petrol|
| hyundai|petrol|
|  maruti|petrol|
| hyundai|diesel|
|  maruti|petrol|
|   honda|petrol|
|     kia|diesel|
| hyundai|petrol|
+--------+------+
only showing top 20 rows



In [29]:
# Filtrando dados com Pandas
tata_cars_pandas = df[df['brand'] == 'tata']
print(tata_cars_pandas.head())

expensive_cars_pandas = df[df['price'] > 10000]
print(expensive_cars_pandas.head())

# Filtrando linhas com múltiplas condições
tata_expensive_cars_pandas = df[(df['brand'] == 'tata') & (df['price'] > 10000)]
print(tata_expensive_cars_pandas.head())


   brand    model transmission  age    fuel  engine       km  owner  \
2   tata  harrier       manual  2.0  diesel  1956.0  27990.0    1.0   
6   tata    punch       manual  2.0  petrol  1199.0    450.0    1.0   
7   tata    nexon    automatic  4.0  diesel     NaN  59866.0    1.0   
12  tata    tiago       manual  1.0  petrol  1199.0   6676.0    1.0   
21  tata    tigor       manual  1.0  petrol  1199.0  12096.0    1.0   

        price location  mileage  power  seats type  
2   1489000.0      NaN      NaN    NaN    NaN  NaN  
6    813000.0      NaN      NaN    NaN    NaN  NaN  
7    728000.0      NaN      NaN    NaN    NaN  NaN  
12   536000.0      NaN      NaN    NaN    NaN  NaN  
21   590000.0      NaN      NaN    NaN    NaN  NaN  
      brand     model transmission  age    fuel  engine       km  owner  \
0  mahindra      thar       manual  4.0  diesel  2184.0  11003.0    1.0   
1   hyundai     verna       manual  6.0  petrol  1591.0  66936.0    1.0   
2      tata   harrier       ma

In [30]:
# Filtrando dados com PySpark
tata_cars_pyspark = df_pyspark.filter(df_pyspark['brand'] == 'honda')
tata_cars_pyspark.show()

expensive_cars_pyspark = df_pyspark.filter(df_pyspark['price'] > 10000)
expensive_cars_pyspark.show()

# Filtrando linhas com múltiplas condições
tata_expensive_cars_pyspark = df_pyspark.filter((df_pyspark['brand'] == 'honda') & (df_pyspark['price'] > 10000))
tata_expensive_cars_pyspark.show()

+-----+-----+------------+---+------+------+--------+-----+---------+--------+-------+-----+-----+----+
|brand|model|transmission|age|  fuel|engine|      km|owner|    price|location|mileage|power|seats|type|
+-----+-----+------------+---+------+------+--------+-----+---------+--------+-------+-----+-----+----+
|honda| city|   automatic|1.0|petrol|1498.0|  5061.0|  1.0|1227000.0|    NULL|   NULL| NULL| NULL|NULL|
|honda| wr-v|      manual|3.0|petrol|1199.0| 44787.0|  1.0| 796000.0|    NULL|   NULL| NULL| NULL|NULL|
|honda|amaze|      manual|2.0|petrol|1199.0| 15935.0|  1.0| 699000.0|    NULL|   NULL| NULL| NULL|NULL|
|honda| city|   automatic|7.0|petrol|1497.0| 65434.0|  2.0| 787000.0|    NULL|   NULL| NULL| NULL|NULL|
|honda| city|   automatic|7.0|petrol|1497.0| 74671.0|  1.0| 832000.0|    NULL|   NULL| NULL| NULL|NULL|
|honda| city|   automatic|8.0|petrol|1497.0| 61033.0|  2.0| 572000.0|    NULL|   NULL| NULL| NULL|NULL|
|honda|amaze|      manual|4.0|petrol|1199.0| 48004.0|  2.0| 4930

In [33]:
# Agregação com Pandas
# Por marca e calcular a média do preço
mean_price_by_brand_pandas = df.groupby('brand')['price'].mean()
print(mean_price_by_brand_pandas)

# Por marca e contar o número de carros
count_cars_by_brand_pandas = df.groupby('brand')['brand'].count()
print(count_cars_by_brand_pandas)

brand
ambassador       1.350000e+05
audi             2.278768e+06
bentley          5.550000e+06
bmw              2.520764e+06
chevrolet        3.044463e+05
datsun           3.232446e+05
fiat             3.269286e+05
force            8.750000e+05
ford             6.626805e+05
honda            6.034868e+05
hyundai          5.712369e+05
isuzu            1.464923e+06
jaguar           2.879663e+06
jeep             1.688935e+06
kia              1.333359e+06
land             3.618103e+06
land rover       3.633041e+06
lexus            4.829444e+06
mahindra         8.335125e+05
maruti           4.942791e+05
maserati         6.100000e+06
mercedes-amg     5.100000e+06
mercedes-benz    2.445811e+06
mg               1.587085e+06
mini             2.489233e+06
mitsubishi       1.105889e+06
nissan           5.568022e+05
porsche          4.457939e+06
renault          4.980811e+05
skoda            7.823020e+05
smart            3.000000e+05
tata             6.824418e+05
toyota           1.210682e+06
volk

In [32]:
# Agregação com PySpark
# Por marca e calcular a média do preço
mean_price_by_brand_pyspark = df_pyspark.groupBy('brand').agg({'price': 'avg'})
mean_price_by_brand_pyspark.show()

# Por marca e contar o número de carros
count_cars_by_brand_pyspark = df_pyspark.groupBy('brand').count()
count_cars_by_brand_pyspark.show()

+----------+------------------+
|     brand|        avg(price)|
+----------+------------------+
|    jaguar|2879663.1578947366|
|land rover|3633040.8163265307|
|  mahindra| 833512.5161290322|
|      tata| 682441.7910447761|
|mitsubishi| 1105888.888888889|
|     lexus| 4829444.444444444|
|      NULL|          976000.0|
|    toyota| 1210682.059447983|
|        mg|1587084.7457627119|
|      audi| 2278768.149882904|
|    datsun| 323244.6483180428|
|       bmw| 2520763.688760807|
|      jeep|1688935.0649350649|
|   bentley|         5550000.0|
|     skoda| 782302.0358306189|
|   hyundai| 571236.9320704455|
|      ford| 662680.5492851768|
|    maruti|494279.10235773126|
|ambassador|          135000.0|
|       kia|1333359.3896713615|
+----------+------------------+
only showing top 20 rows

+----------+-----+
|     brand|count|
+----------+-----+
|    jaguar|   95|
|land rover|   49|
|  mahindra| 1550|
|      tata| 1206|
|mitsubishi|   27|
|     lexus|    9|
|      NULL|    1|
|    toyota| 141

In [34]:
# Junções (Joins) com Pandas
df2 = pd.DataFrame({'brand': ['tata', 'honda', 'maruti'], 'origin': ['India', 'Japan', 'India']})

# Inner Join
merged_df_pandas = pd.merge(df, df2, on='brand', how='inner')
print(merged_df_pandas.head())

# Left Join
merged_df_pandas = pd.merge(df, df2, on='brand', how='left')
print(merged_df_pandas.head())

# Right Join
merged_df_pandas = pd.merge(df, df2, on='brand', how='right')
print(merged_df_pandas.head())

# Outer Join
merged_df_pandas = pd.merge(df, df2, on='brand', how='outer')
print(merged_df_pandas.head())

   brand    model transmission  age    fuel  engine       km  owner  \
0   tata  harrier       manual  2.0  diesel  1956.0  27990.0    1.0   
1  honda     city    automatic  1.0  petrol  1498.0   5061.0    1.0   
2  honda     wr-v       manual  3.0  petrol  1199.0  44787.0    1.0   
3   tata    punch       manual  2.0  petrol  1199.0    450.0    1.0   
4   tata    nexon    automatic  4.0  diesel     NaN  59866.0    1.0   

       price location  mileage  power  seats type origin  
0  1489000.0      NaN      NaN    NaN    NaN  NaN  India  
1  1227000.0      NaN      NaN    NaN    NaN  NaN  Japan  
2   796000.0      NaN      NaN    NaN    NaN  NaN  Japan  
3   813000.0      NaN      NaN    NaN    NaN  NaN  India  
4   728000.0      NaN      NaN    NaN    NaN  NaN  India  
      brand     model transmission  age    fuel  engine       km  owner  \
0  mahindra      thar       manual  4.0  diesel  2184.0  11003.0    1.0   
1   hyundai     verna       manual  6.0  petrol  1591.0  66936.0    1

In [35]:
# Junções (Joins) com PySpark
# Convertendo o DataFrame Pandas para DataFrame PySpark
df2_pyspark = spark.createDataFrame(df2)

# Inner Join
merged_df_pyspark = df_pyspark.join(df2_pyspark, 'brand', 'inner')
merged_df_pyspark.show()

# Left Join
merged_df_pyspark = df_pyspark.join(df2_pyspark, 'brand', 'left')
merged_df_pyspark.show()

# Right Join
merged_df_pyspark = df_pyspark.join(df2_pyspark, 'brand', 'right')
merged_df_pyspark.show()

# Outer Join
merged_df_pyspark = df_pyspark.join(df2_pyspark, 'brand', 'outer')
merged_df_pyspark.show()

+-----+---------+------------+---+------+------+-------+-----+---------+-------------+-------+-----+-----+---------+------+
|brand|    model|transmission|age|  fuel|engine|     km|owner|    price|     location|mileage|power|seats|     type|origin|
+-----+---------+------------+---+------+------+-------+-----+---------+-------------+-------+-----+-----+---------+------+
| tata|    tigor|      manual|2.0|petrol|  NULL|82009.0|  2.0| 392000.0|  maharashtra|   NULL| NULL|  5.0|    sedan| India|
| tata|    tiago|      manual|3.0|petrol|  NULL| 5288.0|  1.0| 623000.0|  maharashtra|   NULL| NULL|  5.0|hatchback| India|
| tata|  harrier|   automatic|3.0|diesel|  NULL|64784.0|  1.0|1986000.0|  maharashtra|   NULL| NULL|  5.0|      suv| India|
| tata|    tiago|   automatic|3.0|petrol|  NULL|15606.0|  2.0| 785000.0|    telangana|   NULL| NULL|  5.0|hatchback| India|
| tata|    tigor|   automatic|4.0|petrol|  NULL|16841.0|  1.0|1012000.0|  maharashtra|   NULL| NULL|  5.0|    sedan| India|
| tata|t

In [40]:
# Novas colunas com Pandas
df['nova_coluna_pandas'] = df['price'] * 2
print(df.head())

df['combinando_coluna_pandas'] = df['brand'] + '-' + df['fuel']
print(df.head())

      brand     model transmission  age    fuel  engine       km  owner  \
0  mahindra      thar       manual  4.0  diesel  2184.0  11003.0    1.0   
1   hyundai     verna       manual  6.0  petrol  1591.0  66936.0    1.0   
2      tata   harrier       manual  2.0  diesel  1956.0  27990.0    1.0   
3     honda      city    automatic  1.0  petrol  1498.0   5061.0    1.0   
4      ford  ecosport       manual  3.0  diesel  1498.0  23480.0    1.0   

       price location  mileage  power  seats type  new_column_pandas  \
0  1231000.0      NaN      NaN    NaN    NaN  NaN          2462000.0   
1   786000.0      NaN      NaN    NaN    NaN  NaN          1572000.0   
2  1489000.0      NaN      NaN    NaN    NaN  NaN          2978000.0   
3  1227000.0      NaN      NaN    NaN    NaN  NaN          2454000.0   
4   887000.0      NaN      NaN    NaN    NaN  NaN          1774000.0   

  combined_column_pandas  nova_coluna_pandas combinando_coluna_pandas  
0        mahindra-diesel           2462000.0

In [41]:
# Novas colunas com PySpark
df_pyspark = df_pyspark.withColumn('nova_coluna_pyspark', df_pyspark['price'] * 2)
df_pyspark.show()

df_pyspark = df_pyspark.withColumn('combinando_coluna_pyspark', df_pyspark['brand'] + '-' + df_pyspark['fuel'])
df_pyspark.show()

+--------+--------+------------+----+------+------+-------+-----+---------+--------+-------+-----+-----+----+------------------+-----------------------+-------------------+
|   brand|   model|transmission| age|  fuel|engine|     km|owner|    price|location|mileage|power|seats|type|new_column_pyspark|combined_column_pyspark|nova_coluna_pyspark|
+--------+--------+------------+----+------+------+-------+-----+---------+--------+-------+-----+-----+----+------------------+-----------------------+-------------------+
|mahindra|    thar|      manual| 4.0|diesel|2184.0|11003.0|  1.0|1231000.0|    NULL|   NULL| NULL| NULL|NULL|         2462000.0|                   NULL|          2462000.0|
| hyundai|   verna|      manual| 6.0|petrol|1591.0|66936.0|  1.0| 786000.0|    NULL|   NULL| NULL| NULL|NULL|         1572000.0|                   NULL|          1572000.0|
|    tata| harrier|      manual| 2.0|diesel|1956.0|27990.0|  1.0|1489000.0|    NULL|   NULL| NULL| NULL|NULL|         2978000.0|       

In [42]:
# Ordenação com Pandas
df_sorted_ascending_pandas = df.sort_values('price')
print(df_sorted_ascending_pandas.head())

df_sorted_descending_pandas = df.sort_values('price', ascending=False)
print(df_sorted_descending_pandas.head())

        brand    model transmission   age    fuel  engine        km  owner  \
16400  maruti  wagon r       manual  21.0  petrol   998.0   80000.0    NaN   
4513     tata     nano       manual  13.0  petrol   624.0   65000.0    2.0   
4429   maruti      800       manual  20.0  petrol   796.0   12000.0    2.0   
5930   maruti      zen       manual  26.0  petrol   993.0   95150.0    3.0   
22452  maruti     alto       manual  17.0  petrol   796.0  110000.0    NaN   

         price location  mileage  power  seats type  new_column_pandas  \
16400  40000.0      NaN     18.9   67.1    5.0  NaN            80000.0   
4513   44000.0     pune     26.0   35.0    4.0  NaN            88000.0   
4429   45000.0   jaipur     16.1   37.0    4.0  NaN            90000.0   
5930   45000.0   jaipur     17.3   60.0    5.0  NaN            90000.0   
22452  45000.0      NaN     19.7   46.3    5.0  NaN            90000.0   

      combined_column_pandas  nova_coluna_pandas combinando_coluna_pandas  
16400     

In [43]:
# Ordenação com PySpark
df_sorted_ascending_pyspark = df_pyspark.orderBy('price')
df_sorted_ascending_pyspark.show()

df_sorted_descending_pyspark = df_pyspark.orderBy('price', ascending=False)
df_sorted_descending_pyspark.show()

+-------+-------+------------+----+------+------+--------+-----+-----------------+---------+-------+-----+-----+----+------------------+-----------------------+-------------------+-------------------------+
|  brand|  model|transmission| age|  fuel|engine|      km|owner|            price| location|mileage|power|seats|type|new_column_pyspark|combined_column_pyspark|nova_coluna_pyspark|combinando_coluna_pyspark|
+-------+-------+------------+----+------+------+--------+-----+-----------------+---------+-------+-----+-----+----+------------------+-----------------------+-------------------+-------------------------+
| maruti|wagon r|      manual|21.0|petrol| 998.0| 80000.0| NULL|          40000.0|     NULL|   18.9| 67.1|  5.0|NULL|           80000.0|                   NULL|            80000.0|                     NULL|
|   tata|   nano|      manual|13.0|petrol| 624.0| 65000.0|  2.0|          44000.0|     pune|   26.0| 35.0|  4.0|NULL|           88000.0|                   NULL|            